# 1. Import bibliotek

In [1]:
import pickle
import pandas as pd
from tqdm import tqdm
import os
import cv2
import numpy as np

# 2. Podział danych

In [3]:
def split_video(input_folder_video_path: str, 
                output_folder_video_path: str,
                original_metadata_path: str,
                category: str,
                HEIGHT = 224,
                WIDTH = 224) -> None :
    
    data = pickle.load(open(original_metadata_path, "rb"))

    metadata = {'video_name' : [],
                'action_idx' : [],
                'category' : [],
                'frames': [],
                'resize': [],
                'source_video': []}
    
    files = os.listdir(input_folder_video_path)


    for file in tqdm(files):
        vid = file.split('.')[0]
        # for idx in tqdm(data['gttubes'][f"{category}/{vid}"]):
        for idx in tqdm(data['gttubes'][f"{category}/{vid}"]):
            # print(idx)
            for i, item in enumerate(data['gttubes'][f"{category}/{vid}"][idx]):
                # print(f"{(i+1):04d}")

                max_bbox = np.max(abs(item[:,1:3] - item[:,3:]),axis=0)
                flag = np.any(max_bbox > 224)

                if flag:
                    # print(f"{output_folder_video_path}/{idx:02d}_{(i+1):04d}_{file}")
                    height = max_bbox[1]
                    width = max_bbox[0]
                else:
                    height = HEIGHT
                    width = WIDTH

                video_capture = cv2.VideoCapture(f"{input_folder_video_path}/{file}")
                fps = int(video_capture.get(cv2.CAP_PROP_FPS))
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(f"{output_folder_video_path}/{idx:02d}_{(i+1):04d}_{file}", fourcc, fps, (WIDTH, HEIGHT))

                resize_flag = None

                for frames_data in item:
                    # print(frames_data)

                    roi = None
                    
                    video_capture.set(cv2.CAP_PROP_POS_FRAMES, frames_data[0])
                    
                    # Odczytanie klatki
                    ret, frame = video_capture.read()
                    if not ret:
                        break
                    
                    # Zaznaczenie bounding boxa na nagraniu
                    x1, y1, x2, y2 = frames_data[1:]

                    add_y = height - abs(y1 - y2)
                    add_x = width - abs(x1 - x2)

                    if add_y % 2 == 0:
                        y1 -= add_y / 2
                        y2 += add_y / 2
                    else:
                        y1 -= (add_y - 1) / 2
                        y2 += (add_y - 1) / 2 + 1
               
                    if add_x % 2 == 0:
                        x1 -= add_x / 2
                        x2 += add_x / 2
                    else:
                        x1 -= (add_x - 1) / 2
                        x2 += (add_x - 1) / 2 + 1
                    
                    if flag:
                        roi = frame[abs(int(y1)):abs(int(y2)), abs(int(x1)):abs(int(x2))]
                        try:
                            roi = cv2.resize(roi, (WIDTH, HEIGHT))
                        except:
                            print(f"[DEBUG]: {roi} {roi.shape} {int(y1),int(y2), int(x1),int(x2)} {frame[0]}")
                        resize_flag = True
                        # print(roi)
                    else:
                        roi = frame[int(y1):int(y2), int(x1):int(x2)]
                        resize_flag = False

                    out.write(roi)

                video_capture.release()
                out.release()


                metadata['video_name'].append(f"{idx:02d}_{(i+1):04d}_{file}")
                metadata['action_idx'].append(idx)
                metadata['category'].append(category)
                metadata['frames'].append(len(item))
                metadata['resize'].append(resize_flag)
                metadata['source_video'].append(file)

    df_metadata = pd.DataFrame(metadata)
    df_metadata.to_csv(f"metadata_{category}.csv", index=False)

## 2.1. Podział video - baskteball

In [4]:
category = "basketball"
input_folder_video_path = "../data/basketball"
output_folder_video_path = "../datasets_raw/basketball"
original_metadata_path = "../multisports_GT.pkl"

split_video(input_folder_video_path,
            output_folder_video_path,
            original_metadata_path,
            category=category)

100%|██████████| 526/526 [20:41<00:00,  2.36s/it]


## 2.2. Podział video - aerobic_gymnasticsm

In [5]:
category = "aerobic_gymnastics"
input_folder_video_path = "../data/aerobic_gymnastics"
output_folder_video_path = "../datasets_raw/aerobic_gymnastics"
original_metadata_path = "../multisports_GT.pkl"

split_video(input_folder_video_path,
            output_folder_video_path,
            original_metadata_path,
            category=category)

100%|██████████| 537/537 [23:56<00:00,  2.67s/it]


## 2.3. Podział video - football

In [6]:
category = "football"
input_folder_video_path = "../data/football"
output_folder_video_path = "../datasets_raw/football"
original_metadata_path = "../multisports_GT.pkl"

split_video(input_folder_video_path,
            output_folder_video_path,
            original_metadata_path,
            category=category)

100%|██████████| 534/534 [19:33<00:00,  2.20s/it]


## 2.4. Podział video - volleyball

In [7]:
category = "volleyball"
input_folder_video_path = "../data/volleyball"
output_folder_video_path = "../datasets_raw/volleyball"
original_metadata_path = "../multisports_GT.pkl"

split_video(input_folder_video_path,
            output_folder_video_path,
            original_metadata_path,
            category=category)

100%|██████████| 532/532 [12:11<00:00,  1.38s/it]
